# GeoAI Testing Demo

This notebook demonstrates the automated testing functionality for the GeoAI package. It shows how to run tests and use the dummy datasets for development and testing.

## 1. Import Required Libraries

In [ ]:
import sys
import os
import unittest
import numpy as np
import rasterio
import geopandas as gpd
from pathlib import Path

# Import geoai modules
import geoai
from geoai import utils, classify, extract, download, segment

## 2. Verify Test Data Creation

In [ ]:
# Check if test data exists
test_data_dir = "tests/data"

if os.path.exists(test_data_dir):
    print(f"Test data directory exists: {test_data_dir}")

    # List all test files
    test_files = os.listdir(test_data_dir)
    print(f"\nTest files: {test_files}")

    # Check specific files
    test_raster = os.path.join(test_data_dir, "test_raster_rgb.tif")
    test_vector = os.path.join(test_data_dir, "test_polygons.geojson")

    if os.path.exists(test_raster):
        with rasterio.open(test_raster) as src:
            print(f"\nTest raster info:")
            print(f"  Shape: {src.height} x {src.width}")
            print(f"  Bands: {src.count}")
            print(f"  CRS: {src.crs}")
            print(f"  Bounds: {src.bounds}")

    if os.path.exists(test_vector):
        gdf = gpd.read_file(test_vector)
        print(f"\nTest vector info:")
        print(f"  Number of features: {len(gdf)}")
        print(f"  CRS: {gdf.crs}")
        print(f"  Columns: {list(gdf.columns)}")
else:
    print(f"Test data directory not found: {test_data_dir}")
    print("Run the create_test_data.py script first!")

## 3. Test Core GeoAI Functionality

In [ ]:
# Test Map creation
print("Testing Map creation...")
try:
    m = geoai.Map()
    print("✓ Map created successfully")
    print(f"  Map type: {type(m)}")
except Exception as e:
    print(f"✗ Map creation failed: {e}")

# Test MapLibre creation
print("\nTesting MapLibre creation...")
try:
    ml = geoai.MapLibre()
    print("✓ MapLibre created successfully")
    print(f"  MapLibre type: {type(ml)}")
except Exception as e:
    print(f"✗ MapLibre creation failed: {e}")

## 4. Test Device Detection

In [ ]:
# Test device detection
print("Testing device detection...")
try:
    device = utils.get_device()
    print(f"✓ Device detected: {device}")
    print(f"  Device type: {device.type}")

    # Check if PyTorch is working
    import torch

    test_tensor = torch.tensor([1, 2, 3]).to(device)
    print(f"  Test tensor on device: {test_tensor}")

except Exception as e:
    print(f"✗ Device detection failed: {e}")

## 5. Test Data Loading with Dummy Datasets

In [ ]:
# Test loading raster data
test_raster = "tests/data/test_raster_rgb.tif"

if os.path.exists(test_raster):
    print("Testing raster data loading...")

    try:
        # Load with rasterio
        with rasterio.open(test_raster) as src:
            data = src.read()
            print(f"✓ Raster loaded successfully")
            print(f"  Data shape: {data.shape}")
            print(f"  Data type: {data.dtype}")
            print(f"  Value range: {data.min()} - {data.max()}")

        # Test basic statistics
        print(f"\nBasic statistics:")
        for i in range(data.shape[0]):
            band_data = data[i]
            print(
                f"  Band {i+1}: mean={band_data.mean():.2f}, std={band_data.std():.2f}"
            )

    except Exception as e:
        print(f"✗ Raster loading failed: {e}")
else:
    print(f"Test raster not found: {test_raster}")

## 6. Test Vector Data Loading

In [ ]:
# Test loading vector data
test_vector = "tests/data/test_polygons.geojson"

if os.path.exists(test_vector):
    print("Testing vector data loading...")

    try:
        gdf = gpd.read_file(test_vector)
        print(f"✓ Vector data loaded successfully")
        print(f"  Number of features: {len(gdf)}")
        print(f"  Geometry types: {gdf.geometry.type.unique()}")
        print(f"  Total area: {gdf.geometry.area.sum():.6f}")

        # Display first few features
        print(f"\nFirst 3 features:")
        print(gdf.head(3))

    except Exception as e:
        print(f"✗ Vector loading failed: {e}")
else:
    print(f"Test vector not found: {test_vector}")

## 7. Run Unit Tests Programmatically

In [ ]:
# Run specific test modules
print("Running unit tests programmatically...")


def run_test_module(module_name):
    """Run tests for a specific module."""
    try:
        # Import the test module
        sys.path.insert(0, "tests")
        test_module = __import__(module_name)

        # Create test suite
        loader = unittest.TestLoader()
        suite = loader.loadTestsFromModule(test_module)

        # Run tests
        runner = unittest.TextTestRunner(verbosity=2)
        result = runner.run(suite)

        print(f"\n{module_name} Results:")
        print(f"  Tests run: {result.testsRun}")
        print(f"  Failures: {len(result.failures)}")
        print(f"  Errors: {len(result.errors)}")
        print(
            f"  Success rate: {((result.testsRun - len(result.failures) - len(result.errors)) / result.testsRun * 100):.1f}%"
        )

        return result.wasSuccessful()

    except Exception as e:
        print(f"✗ Failed to run {module_name}: {e}")
        return False


# Test individual modules
test_modules = ["test_utils", "test_classify", "test_download"]

for module in test_modules:
    print(f"\n{'='*50}")
    print(f"Testing {module}")
    print(f"{'='*50}")
    success = run_test_module(module)
    if success:
        print(f"✓ {module} tests passed")
    else:
        print(f"✗ {module} tests had issues")

## 8. Test Image Chip Data

In [ ]:
# Test image chips (for training)
chips_dir = "tests/data/chips"

if os.path.exists(chips_dir):
    print("Testing image chips...")

    chip_files = [f for f in os.listdir(chips_dir) if f.endswith(".tif")]
    mask_files = [f for f in os.listdir(chips_dir) if f.endswith(".png")]

    print(f"Found {len(chip_files)} image chips and {len(mask_files)} masks")

    if chip_files:
        # Load first chip
        from PIL import Image

        chip_path = os.path.join(chips_dir, chip_files[0])
        mask_path = os.path.join(chips_dir, mask_files[0]) if mask_files else None

        try:
            chip_img = Image.open(chip_path)
            print(f"✓ Loaded chip: {chip_img.size} pixels, mode: {chip_img.mode}")

            if mask_path:
                mask_img = Image.open(mask_path)
                print(f"✓ Loaded mask: {mask_img.size} pixels, mode: {mask_img.mode}")

                # Convert to numpy for analysis
                mask_array = np.array(mask_img)
                unique_values = np.unique(mask_array)
                print(f"  Mask classes: {unique_values}")

        except Exception as e:
            print(f"✗ Failed to load chips: {e}")
else:
    print(f"Chips directory not found: {chips_dir}")

## 9. Test Configuration Loading

In [ ]:
# Test configuration file
config_file = "tests/data/test_config.json"

if os.path.exists(config_file):
    print("Testing configuration loading...")

    try:
        import json

        with open(config_file, "r") as f:
            config = json.load(f)

        print(f"✓ Configuration loaded successfully")
        print(f"  Config keys: {list(config.keys())}")

        if "model_config" in config:
            print(f"  Model config: {config['model_config']}")

        if "data_config" in config:
            print(f"  Data config: {config['data_config']}")

    except Exception as e:
        print(f"✗ Configuration loading failed: {e}")
else:
    print(f"Configuration file not found: {config_file}")

## 10. Summary and Next Steps

In [ ]:
print("\n" + "=" * 60)
print("TESTING SUMMARY")
print("=" * 60)

print("\n✓ Automated testing framework successfully added to GeoAI package!")

print("\nWhat was created:")
print("  • Comprehensive test suite with 33 unit tests")
print("  • Dummy datasets for testing (rasters, vectors, image chips)")
print("  • Test modules for all major components:")
print("    - test_geoai.py (core functionality)")
print("    - test_utils.py (utility functions)")
print("    - test_classify.py (classification module)")
print("    - test_download.py (data download module)")
print("    - test_segment.py (segmentation module)")
print("    - test_extract.py (feature extraction module)")
print("  • Test data creation script (create_test_data.py)")
print("  • This demonstration notebook")

print("\nHow to run tests:")
print("  • All tests: python -m unittest discover tests/ -v")
print("  • Specific module: python -m unittest tests.test_utils -v")
print(
    "  • Individual test: python -m unittest tests.test_utils.TestUtilsFunctions.test_get_device -v"
)

print("\nTest coverage includes:")
print("  • Package imports and module availability")
print("  • Core class instantiation (Map, MapLibre)")
print("  • Device detection and PyTorch integration")
print("  • Function signature validation")
print("  • Error handling with invalid inputs")
print("  • Data loading and basic operations")

print("\nNext steps for expanding tests:")
print("  • Add integration tests with real data")
print("  • Add performance benchmarks")
print("  • Add tests for machine learning model training")
print("  • Add tests for visualization components")
print("  • Set up continuous integration (CI) testing")